# Analisi - Grafi

In [1]:
import pickle
import json

import geopandas as gpd
import networkx as nx
from networkx.algorithms.approximation import steiner_tree
import osmnx as ox

from my_paths import *
import Graph_functions

## Analisi Automatica

Tramite la funzione Graph_functions.auto_analysis_poi() è possibile eseguire in automatico la ricreazione del grafo stradale con il dizionario dei pesi desiderato.
Si può ora caricaricare da file il dizionario dei pesi che abbiamo pre-stabilito.
Aggiunta possibilità di inserire come chiave "default" nel "custom_weights" in modo da dare un valore di base quando non viene trovata una strada. Grazie a questo è possibile utilizzare un dizionario del tipo:
```python
{"default": 1}
```
In questo modo a tutte le strade verrà assegnato il peso = 1.
Si possono inserire i vari percorsi di salvataggio dei file pickle (grafo) e geojson(geoDataFrame).
Richiede una struttura a liste di dizionari di geoDataFrame per "poi" del tipo:
```python
gdf_list = [
    {"gdf":  gpd.read_file(PATH_DEL_GEOJSON),
     "tipo": "tipo che si vuole assegnare agli archi che connettono il gdf al grafo",
     "attr": {"nome_attributo": "valore_attributo"} # Opzionale, di solito non ci serve
     }
]
```

## Sport e tempo libero

Utilizzeremo i seguenti dati:
1. **Parchi**
2. **Impianti sportivi**
3. **Aree gioco**
Saranno i nostri "poi", cioè "Point of Interest" che verranno aggiunti al Grafo della rete Ciclabile/Stradale per la ricerca di percorsi

Con pesi

In [2]:
with open(PATH_CUSTOM_WEIGHTS_STRADE_RAW, 'r') as f:
    custom_weights = json.load(f)
gdf_list = [
    {"gdf":  gpd.read_file(PATH_PARCHI_CLEAN),
     "tipo": "parchi"}
]
custom_weights["cycleway"] = 0
custom_weights["unclassified"] = 99
custom_weights["footway"] = 1.3
Graph_functions.auto_analysis_poi(gdf_list, custom_weights,
                                  PATH_SPORT_TEMPO_LIBERO_ANALISI_CLEAN)

Senza pesi

In [3]:
gdf_list = [
    {"gdf":  gpd.read_file(PATH_PARCHI_CLEAN),
     "tipo": "parchi"}
]
Graph_functions.auto_analysis_poi(gdf_list, {"cycleway": 0, "default": 1},
                                  "../Data/Clean/Analisi/sport_tempo_libero_clean_nopesi.geojson")

### Esempio di statisiche per confrontare 2 grafi: uno con i nostri pesi e l'altro fatto girare con pesi = 1

In [4]:
gdf_pesi = gpd.read_file(PATH_SPORT_TEMPO_LIBERO_ANALISI_CLEAN)
gdf_no_pesi = gpd.read_file("../Data/Clean/Analisi/sport_tempo_libero_clean_nopesi.geojson")

In [5]:
gdf_pesi_mod = gdf_pesi[gdf_pesi["artificial"] != True]
gdf_no_pesi_mod = gdf_no_pesi[gdf_no_pesi["artificial"] != True]

print(gdf_pesi_mod["poi"].unique())

gdf = gpd.read_file(PATH_PARCHI_CLEAN)
geometry_parchi = gdf.union_all()
list_gdf = [gdf_pesi_mod, gdf_no_pesi_mod]
for i in range(len(list_gdf)):
    gdf_senza_parchi = []
    for idx, row in list_gdf[i].iterrows():
        if not geometry_parchi.contains(row.geometry):
            gdf_senza_parchi.append(row)
    list_gdf[i] = gpd.GeoDataFrame(gdf_senza_parchi, crs=CRS_GRAD)

gdf_pesi_mod = list_gdf[0]
gdf_no_pesi_mod = list_gdf[1]

gdf_pesi_mod.to_file("../Data/Clean/Analisi/sport_senza_parchi_con_pesi.geojson")
gdf_no_pesi_mod.to_file("../Data/Clean/Analisi/sport_senza_parchi_senza_pesi.geojson")

gdf_pesi_mod_most_highways = gdf_pesi_mod.to_crs(CRS_METR).groupby("highway").agg({
    "length": lambda x: sum(x/1000)
    }).sort_values(by="length", ascending=False)

gdf_no_pesi_mod_most_highways = gdf_no_pesi_mod.to_crs(CRS_METR).groupby("highway").agg({
    "length": lambda x: sum(x/1000)
}).sort_values(by="length", ascending=False)

[False]


In [ ]:
print("----------PESI-------------")
print(gdf_pesi_mod_most_highways)
print(f"Tot: {gdf_pesi_mod_most_highways["length"].sum()}")

print("----------NO PESI----------")
print(gdf_no_pesi_mod_most_highways)
print(f"Tot: {gdf_no_pesi_mod_most_highways["length"].sum()}")